In [1]:
import copy

# 外皮情報LV3からLV4へのコンバート

## I. 関数

### 1. 間仕切り床

#### 1) 床の仕様

| 番号 | 材料 | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> | 厚さ, m |
|---|---|---|---|---|
| 1 | 合板 | 0.16 | 720.0 | 0.024 |

In [2]:
def get_inner_floor_spec():
    
    return {
        'layers' : [
            {
                'name'                            : 'plywood',
                'thermal_resistance_input_method' : 'conductivity',
                'thermal_conductivity'            : 0.16,
                'volumetric_specific_heat'        : 720.0,
                'thickness'                       : 0.024,
            }
        ]
    }

#### 2) 方位が下面である外皮の部位の面積の合計

$$A_{evlp,down,MR} = \sum_i{A_{evlp,down,MR,i}}$$ 

$$A_{evlp,down,OR} = \sum_i{A_{evlp,down,OR,i}}$$ 

$$A_{evlp,down,NR} = \sum_i{A_{evlp,down,NR,i}}$$ 

$A_{evlp,down,MR}$：主たる居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,OR}$：その他の居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,NR}$：非居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,MR,i}$：主たる居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  
$A_{evlp,down,OR,i}$：その他の居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  
$A_{evlp,down,NR,i}$：非居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  

In [3]:
def get_downward_envelope_total_area(envelope):

    parts = []

    if 'general_parts' in envelope: parts.extend(envelope['general_parts']) 
    
    if 'windows' in envelope: parts.extend(envelope['windows'])
        
    if 'doors' in envelope: parts.extend(envelope['doors'])
    
    def f(space_type):
        return sum(part['area'] for part in parts
                   if part['space_type'] == space_type and (part['direction'] == 'bottom' or part['direction'] == 'downward'))

    return f('main_occupant_room'), f('other_occupant_room'), f('non_occupant_room')

#### 3) 土間床等の中心部の面積の合計

$$A_{ef,MR} = \sum_i {A_{ef,MR,i}}$$ 

$$A_{ef,OR}=\sum_i{A_{ef,OR,i}}$$

$$A_{ef,NR}=\sum_i{A_{ef,NR,i}}$$ 

$$A_{ef,uf}=\sum_i{A_{ef,uf,i}}$$ 

$A_{ef,MR}$：主たる居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,OR}$：その他の居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,NR}$：非居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,MR,i}$：主たる居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$：その他の居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$：非居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部$i$の面積, m<sup>2</sup>  

In [4]:
def get_earthfloor_total_area(envelope):

    parts = []
    
    if 'earthfloor_centers' in envelope: parts.extend(envelope['earthfloor_centers'])
    
    def f(space_type):
        return sum(part['area'] for part in parts
                   if part['space_type'] == space_type)

    return f('main_occupant_room'), f('other_occupant_room'), f('non_occupant_room'), f('underfloor')

#### 4) 間仕切り床の面積の合計

$$
A_{if,MR} = \max \left( 0, A_{MR} - A_{evlp,down,MR} - A_{ef,MR} \right)
$$

$$
A_{if,OR} = \max \left( 0, A_{OR} - A_{evlp,down,OR} - A_{ef,OR} \right)
$$

$$
A_{if,NR} = \max \left( 0, A_A - A_{MR} - A_{OR} - A_{evlp,down,NR} - A_{ef,NR} \right)
$$

$A_{if,MR}$：床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$：床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$：床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{A}$: 床面積の合計, m<sup>2</sup>   
$A_{evlp,down,MR}$：主たる居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,OR}$：その他の居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,NR}$：非居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{ef,MR}$：主たる居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,OR}$：その他の居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,NR}$：非居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  

In [5]:
def get_inner_floor_total_area(a_a, a_mr, a_or,
                               a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr,
                               a_ef_mr, a_ef_or, a_ef_nr):

    a_if_mr = max(0.0, a_mr - a_evlp_down_mr - a_ef_mr)
    a_if_or = max(0.0, a_or - a_evlp_down_or - a_ef_or)
    a_if_nr = max(0.0, a_a - a_mr - a_or - a_evlp_down_nr - a_ef_nr)
    
    return a_if_mr, a_if_or, a_if_nr

#### 5) 間仕切り床の面積の合計（床下側が床下空間）

$A_{if,MR} + A_{if,OR} + A_{if,NR} > 0$の場合

$$A_{if,MR-uf} = A_{ef,uf} \frac{A_{if,MR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$$A_{if,OR-uf} = A_{ef,uf} \frac{A_{if,OR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$$A_{if,NR-uf} = A_{ef,uf} \frac{A_{if,NR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$A_{if,MR} + A_{if,OR} + A_{if,NR} = 0$の場合

$$A_{if,MR-uf}=0$$

$$A_{if,OR-uf}=0$$

$$A_{if,NR-uf}=0$$

$A_{if,MR-uf}$：床上側が主たる居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-uf}$：床上側がその他の居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-uf}$：床上側が非居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{if,MR}$：床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$：床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$：床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  

In [6]:
def get_inner_floor_over_underfloor_total_area(a_if_mr, a_if_or, a_if_nr, a_ef_uf):
    
    if (a_if_mr + a_if_or + a_if_nr) > 0.0:
        a_if_mr_uf = a_ef_uf * a_if_mr / (a_if_mr + a_if_or + a_if_nr)
        a_if_or_uf = a_ef_uf * a_if_or / (a_if_mr + a_if_or + a_if_nr)
        a_if_nr_uf = a_ef_uf * a_if_nr / (a_if_mr + a_if_or + a_if_nr)
    else :
        a_if_mr_uf = 0.0
        a_if_or_uf = 0.0
        a_if_nr_uf = 0.0
    
    return a_if_mr_uf, a_if_or_uf, a_if_nr_uf

#### 6) 間仕切り床の面積の合計（床下側が主たる居室・その他の居室・非居室）

$$ A_{if,MR-OR,NR} = \max{ \left( 0, A_{if,MR}-A_{if,MR-uf} \right) } $$

$$ A_{if,OR-MR,NR} = \max{ \left( 0, A_{if,OR}-A_{if,OR-uf} \right) } $$

$$ A_{if,NR-MR,OR} = \max{ \left( 0, A_{if,NR}-A_{if,NR-uf} \right) } $$

$A_{OR}=0$かつ$A_{NR}=0$の場合

$$A_{if,MR-OR}=0$$

$$A_{if,MR-NR}=0$$

$$A_{if,OR-MR}=0$$

$$A_{if,OR-NR}=0$$

$$A_{if,NR-MR}=0$$

$$A_{if,NR-OR}=0$$

$A_{OR}=0$かつ$A_{NR}\ne0$の場合

$$A_{if,MR-OR}=0$$

$$A_{if,MR-NR}=A_{if,MR-OR,NR}$$

$$A_{if,OR-MR}=0$$

$$A_{if,OR-NR}=0$$

$$A_{if,NR-MR}=A_{if,NR-MR,OR}$$

$$A_{if,NR-OR}=0$$

$A_{OR}\ne0$かつ$A_{NR}=0$の場合

$$A_{if,MR-OR}=A_{if,MR-OR,NR}$$

$$A_{if,MR-NR}=0$$

$$A_{if,NR-MR}=0$$

$$A_{if,NR-OR}=0$$

$$A_{if,OR-MR}=A_{if,OR-MR,NR}$$

$$A_{if,OR-NR}=0$$

$A_{OR}\ne0$かつ$A_{NR}\ne0$の場合

$$A_{if,MR-OR} = \frac{ A_{if,MR-OR,NR} }{ 2 }$$

$$A_{if,MR-NR} = \frac{ A_{if,MR-OR,NR} }{ 2 }$$

$$A_{if,OR-MR} = \frac{ A_{if,OR-MR,NR} }{ 2 }$$

$$A_{if,OR-NR} = \frac{ A_{if,OR-MR,NR} }{ 2 }$$

$$A_{if,NR-MR} = \frac{ A_{if,NR-MR,OR} }{ 2 }$$

$$A_{if,NR-OR} = \frac{ A_{if,NR-MR,OR} }{ 2 }$$

$A_{if,MR-OR,NR}$:床上側が主たる居室に接し床下側がその他の居室又は非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-MR,NR}$:床上側がその他の居室に接し床下側が主たる居室又は非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-MR,OR}$:床上側が非居室に接し床下側が主たる居室又はその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR}$:床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$:床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$:床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-uf}$:床上側が主たる居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-uf}$:床上側がその他の居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-uf}$:床上側が非居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-OR}$:床上側が主たる居室に接し床下側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-NR}$:床上側が主たる居室に接し床下側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-MR}$:床上側がその他の居室に接し床下側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-NR}$:床上側がその他の居室に接し床下側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-MR}$:床上側が非居室に接し床下側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-OR}$:床上側が非居室に接し床下側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  

In [7]:
def get_inner_floor_between_rooms(a_mr, a_or, a_a, a_if_mr, a_if_or, a_if_nr, a_if_mr_uf, a_if_or_uf, a_if_nr_uf):
    
    a_nr = a_a - a_mr - a_or
    
    if a_mr <= 0.0:
        raise ValueError('主たる居室の面積に0又は負の値が指定されました。主たる居室は必ず0より大きい値を指定する必要があります。')
    
    a_if_mr_or_nr = max(0.0, a_if_mr - a_if_mr_uf)
    a_if_or_mr_nr = max(0.0, a_if_or - a_if_or_uf)
    a_if_nr_mr_or = max(0.0, a_if_nr - a_if_nr_uf)

    if a_or <= 0.0 and a_nr <= 0.0:
        a_if_mr_or = 0.0
        a_if_mr_nr = 0.0
        a_if_or_mr = 0.0
        a_if_or_nr = 0.0
        a_if_nr_mr = 0.0
        a_if_nr_or = 0.0
    elif a_or <= 0.0:
        a_if_mr_or = 0.0
        a_if_mr_nr = a_if_mr_or_nr
        a_if_or_mr = 0.0
        a_if_or_nr = 0.0
        a_if_nr_mr = a_if_nr_mr_or
        a_if_nr_or = 0.0
    elif a_nr <= 0.0:
        a_if_mr_or = a_if_mr_or_nr
        a_if_mr_nr = 0.0
        a_if_or_mr = a_if_or_mr_nr
        a_if_or_nr = 0.0
        a_if_nr_mr = 0.0
        a_if_nr_or = 0.0
    else:
        a_if_mr_or = a_if_mr_or_nr / 2
        a_if_mr_nr = a_if_mr_or_nr / 2
        a_if_or_mr = a_if_or_mr_nr / 2
        a_if_or_nr = a_if_or_mr_nr / 2
        a_if_nr_mr = a_if_nr_mr_or / 2
        a_if_nr_or = a_if_nr_mr_or / 2
    
    return a_if_mr_or, a_if_mr_nr, a_if_or_mr, a_if_or_nr, a_if_nr_mr, a_if_nr_or

### 2. 間仕切り壁

#### 1) 壁の仕様

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| せっこうボード | 0.0125 | 0.22 | - | 830.0 |
| 非密閉空気層 | - | - | 0.090 | 0.0 |
| せっこうボード | 0.0125 | 0.22 | - | 830.0 |

In [8]:
def get_inner_wall_spec():
    return {
        'layers' : [
            {
                'name'                            : 'gpb',
                'thermal_resistance_input_method' : 'conductivity',
                'thermal_conductivity'            : 0.22,
                'thickness'                       : 0.0125,
                'volumetric_specific_heat'        : 830.0,
            },
            {
                'name'                            : 'air_layer',
                'thermal_resistance_input_method' : 'resistance',
                'thermal_resistance'              : 0.09,
                'volumetric_specific_heat'        : 0.0,
            },
            {
                'name'                            : 'gpb',
                'thermal_resistance_input_method' : 'conductivity',
                'thermal_conductivity'            : 0.22,
                'thickness'                       : 0.0125,
                'volumetric_specific_heat'        : 830.0,
            },
        ]
    }

#### 2) 方位が水平である外皮の部位の面積の合計

$$ A_{evlp,hzt,MR} = \sum_i{ A_{evlp,hzt,MR,i} } $$

$$ A_{evlp,hzt,OR} = \sum_i{ A_{evlp,hzt,OR,i} } $$

$$ A_{evlp,hzt,NR} = \sum_i{ A_{evlp,hzt,NR,i} } $$

$ A_{evlp,hzt,MR} $:主たる居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$ A_{evlp,hzt,OR} $:その他の居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$ A_{evlp,hzt,NR} $:非居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$ A_{evlp,hzt,MR,i} $:主たる居室に接する方位が水平である外皮の部位$i$の面積, m<sup>2</sup>  
$ A_{evlp,hzt,OR,i} $:その他の居室に接する方位が水平である外皮の部位$i$の面積, m<sup>2</sup>  
$ A_{evlp,hzt,NR,i} $:非居室に接する方位が水平である外皮の部位$i$の面積, m<sup>2</sup>  

In [9]:
def get_horizontal_envelope_total_area(envelope):

    parts = []

    if 'general_parts' in envelope: parts.extend(envelope['general_parts'])
        
    if 'windows' in envelope: parts.extend(envelope['windows'])
        
    if 'doors' in envelope: parts.extend(envelope['doors'])

    def is_horizontal(part):
        d = part['direction']
        return d == 'n' or d == 'ne' or d == 'e' or d == 'se' or \
               d == 's' or d == 'sw' or d == 'w' or d == 'nw' or \
               d == 'horizontal'
        
    def f(space_type):
        return sum(part['area'] for part in parts \
                   if part['space_type'] == space_type and is_horizontal(part))
    
    return f('main_occupant_room'), f('other_occupant_room'), f('non_occupant_room')

#### 3) 間仕切り壁の面積の合計

$$ A_{iw,MR} = \max{ \left( 0, 4 r_{iw,MR} h_{iw} \sqrt{ A_{MR} } - A_{evlp,hzt,MR} \right) } $$

$$ A_{iw,OR} = \max⁡{ \left( 0, 4 r_{iw,OR} h_{iw} \sqrt{ A_{OR} } - A_{evlp,hzt,OR} \right) } $$

$$ A_{iw,NR} = \max⁡{ \left( 0, 4 r_{iw,NR} h_{iw} \sqrt{ A_A - A_{MR} - A_{OR} } - A_{evlp,hzt,NR} \right) } $$

$A_{iw,MR}$:主たる居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,OR}$:その他の居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,NR}$:非居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$r_{iw,MR}$:主たる居室の周長を求めるための形状係数($=1.2$)  
$r_{iw,OR}$:その他の居室の周長を求めるための形状係数($=1.4$)  
$r_{iw,NR}$:非居室の周長を求めるための形状係数($=1.4$)  
$h_{iw}$:階高($=2.8$), m  
$A_{MR}$:主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$:その他の居室の床面積, m<sup>2</sup>  
$A_A$:床面積の合計, m<sup>2</sup>  
$A_{evlp,hzt,MR}$:主たる居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,hzt,OR}$:その他の居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,hzt,NR}$:非居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  

In [10]:
def get_inner_wall_total_area(a_mr, a_or, a_a, a_evlp_hzt_mr, a_evlp_hzt_or, a_evlp_hzt_nr):
    
    r_iw_mr, r_iw_or, r_iw_nr = 1.2, 1.4, 1.4

    h_iw = 2.8
    
    a_iw_mr = max( 0.0, 4 * r_iw_mr * h_iw *            a_mr**0.5 - a_evlp_hzt_mr )
    a_iw_or = max( 0.0, 4 * r_iw_or * h_iw *            a_or**0.5 - a_evlp_hzt_or )
    a_iw_nr = max( 0.0, 4 * r_iw_nr * h_iw * (a_a-a_mr-a_or)**0.5 - a_evlp_hzt_nr )
    
    return a_iw_mr, a_iw_or, a_iw_nr

#### 4) 各用途間の間仕切り壁の面積の合計

$ A_{OR} = 0 $ かつ $ A_A - A_{MR} - A_{OR} = 0 $の場合

$$ A_{iw,MR-OR} = 0 $$

$$ A_{iw,MR-NR} = 0 $$

$$ A_{iw,OR-NR} = 0 $$

$ A_{OR} = 0 $ かつ $ A_A - A_{MR} - A_{OR} \ne 0 $ の場合

$$ A_{iw,MR-OR} = 0 $$

$$ A_{iw,MR-NR} = \frac{ \left( A_{iw,MR} + A_{iw,NR} \right) }{ 2 } $$

$$ A_{iw,OR-NR} = 0 $$

$ A_{OR} \ne 0 $ かつ $ A_A - A_{MR} - A_{OR} = 0 $ の場合

$$ A_{iw,MR-OR} = 0 $$

$$ A_{iw,MR-NR} = 0 $$

$$ A_{iw,OR-NR} = \frac{ \left( A_{iw,OR} + A_{iw,NR} \right) }{ 2 } $$

$ A_{OR} \ne 0 $ かつ $ A_A - A_{MR} - A_{OR} \ne 0 $ の場合

$$ A_{iw,MR-OR} = \max{ \left( 0.0, \frac{ \left( A_{iw,MR} + A_{iw,OR} -A_{iw,NR} \right) }{ 2 } \right) } $$

$$ A_{iw,MR-NR} = \max{ \left( 0.0, \frac{ \left( A_{iw,MR} + A_{iw,NR} -A_{iw,OR} \right) }{ 2 } \right) } $$

$$ A_{iw,OR-NR} = \max{ \left( 0.0, \frac{ \left( A_{iw,OR} + A_{iw,NR} -A_{iw,MR} \right) }{ 2 } \right) } $$

$A_{MR}$:主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$:その他の居室の床面積, m<sup>2</sup>  
$A_A$:床面積の合計, m<sup>2</sup>  
$A_{iw,MR-OR}$:主たる居室とその他の居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,MR-NR}$:主たる居室と非居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,OR-NR}$:その他の居室と非居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,MR}$:主たる居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,OR}$:その他の居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,NR}$:非居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  

In [11]:
def get_inner_wall_total_area_between_rooms(a_mr, a_or, a_a, a_iw_mr, a_iw_or, a_iw_nr):

    a_nr = a_a - a_mr - a_or
    
    if a_mr <= 0.0:
        raise ValueError('主たる居室の面積に0又は負の値が指定されました。主たる居室は必ず0より大きい値を指定する必要があります。')
    
    if a_or <= 0.0 and a_nr <= 0.0:
        a_iw_mr_or = 0.0
        a_iw_mr_nr = 0.0
        a_iw_or_nr = 0.0
    elif a_or <= 0.0:
        a_iw_mr_or = 0.0
        a_iw_mr_nr = (a_iw_mr + a_iw_nr)/2
        a_iw_or_nr = 0.0
    elif a_nr <= 0.0:
        a_iw_mr_or = (a_iw_mr + a_iw_or)/2
        a_iw_mr_nr = 0.0
        a_iw_or_nr = 0.0
    else:
        a_iw_mr_or = max(0.0, (a_iw_mr + a_iw_or - a_iw_nr) / 2)
        a_iw_mr_nr = max(0.0, (a_iw_mr + a_iw_nr - a_iw_or) / 2)
        a_iw_or_nr = max(0.0, (a_iw_or + a_iw_nr - a_iw_mr) / 2)
    
    return a_iw_mr_or, a_iw_mr_nr, a_iw_or_nr

## II. 統合

### 1. 室内床

In [12]:
def make_inner_floors(envelope, common):

    a_mr = common['main_occupant_room_floor_area']
    a_or = common['other_occupant_room_floor_area']
    a_a  = common['total_floor_area']

    a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr = get_downward_envelope_total_area(envelope)
    
    a_ef_mr, a_ef_or, a_ef_nr, a_ef_uf = get_earthfloor_total_area(envelope)

    a_if_mr, a_if_or, a_if_nr = get_inner_floor_total_area(a_a, a_mr, a_or,
                                                           a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr,
                                                           a_ef_mr, a_ef_or, a_ef_nr)

    a_if_mr_uf, a_if_or_uf, a_if_nr_uf = get_inner_floor_over_underfloor_total_area(a_if_mr, a_if_or, a_if_nr, a_ef_uf)    

    a_if_mr_or, a_if_mr_nr, a_if_or_mr, a_if_or_nr, a_if_nr_mr, a_if_nr_or \
    = get_inner_floor_between_rooms(a_mr, a_or, a_a, a_if_mr, a_if_or, a_if_nr, a_if_mr_uf, a_if_or_uf, a_if_nr_uf)

    ifs = []
    
    def append_inner_floor(ifs, name, area, upper_space_type, lower_space_type):
        
        if area > 0.0:
            ifs.append(
                {
                    'name'             : name,
                    'area'             : area,
                    'upper_space_type' : upper_space_type,
                    'lower_space_type' : lower_space_type,
                    'spec'             : get_inner_floor_spec()
                }
            )

    append_inner_floor(ifs, 'inner_floor_main_to_underfloor',  a_if_mr_uf, 'main_occupant_room',  'underfloor'          )
    append_inner_floor(ifs, 'inner_floor_other_to_underfloor', a_if_or_uf, 'other_occupant_room', 'underfloor'          )
    append_inner_floor(ifs, 'inner_floor_non_to_underfloor',   a_if_nr_uf, 'non_occupant_room',   'underfloor'          )
    append_inner_floor(ifs, 'inner_floor_main_to_other',       a_if_mr_or, 'main_occupant_room',  'other_occupant_room' )
    append_inner_floor(ifs, 'inner_floor_main_to_non',         a_if_mr_nr, 'main_occupant_room',  'non_occupant_room'   )
    append_inner_floor(ifs, 'inner_floor_other_to_main',       a_if_or_mr, 'other_occupant_room', 'main_occupant_room'  )
    append_inner_floor(ifs, 'inner_floor_other_to_non',        a_if_or_nr, 'other_occupant_room', 'non_occupant_room'   )
    append_inner_floor(ifs, 'inner_floor_non_to_main',         a_if_nr_mr, 'non_occupant_room',   'main_occupant_room'  )
    append_inner_floor(ifs, 'inner_floor_non_to_other',        a_if_nr_or, 'non_occupant_room',   'other_occupant_room' )
                    
    return ifs

example

In [13]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name' : 'floor',
                'evlp_type' : 'floor',
                'next_space' : 'open_underfloor',
                'direction' : 'bottom',
                'area' : 60.0,
                'space_type': 'main',
                'spec' : 'something'
            },
        ],
        'earthfloor_centers': [
            { 'name': 'other',    'area': 5.0, 'space_type': 'underfloor' },
            { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
        ]
    }
}

make_inner_floors(d['envelope'], d['common'])

[{'name': 'inner_floor_main_to_underfloor',
  'area': 2.5,
  'upper_space_type': 'main_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     'thickness': 0.024}]}},
 {'name': 'inner_floor_other_to_underfloor',
  'area': 5.0,
  'upper_space_type': 'other_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     'thickness': 0.024}]}},
 {'name': 'inner_floor_non_to_underfloor',
  'area': 2.5,
  'upper_space_type': 'non_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     '

### 2. 室内壁

In [14]:
def make_inner_walls(envelope, common):

    a_mr = common['main_occupant_room_floor_area']
    a_or = common['other_occupant_room_floor_area']
    a_a  = common['total_floor_area']
 
    a_ow_mr, a_ow_or, a_ow_nr = get_horizontal_envelope_total_area(envelope)

    a_iw_mr, a_iw_or, a_iw_nr = get_inner_wall_total_area(a_mr, a_or, a_a, a_ow_mr, a_ow_or, a_ow_nr)
        
    a_iw_mr_or, a_iw_mr_nr, a_iw_or_nr = get_inner_wall_total_area_between_rooms(a_mr, a_or, a_a, a_iw_mr, a_iw_or, a_iw_nr)
    
    iws = []

    def append_inner_wall(iws, name, area, space_type_1, space_type_2):
        
        if area > 0.0:
            iws.append(
                {
                    'name'         : name,
                    'area'         : area,
                    'space_type_1' : space_type_1,
                    'space_type_2' : space_type_2,
                    'spec'         : get_inner_wall_spec()
                }
            )

    append_inner_wall(iws, 'inner_wall_main_to_other', a_iw_mr_or, 'main_occupant_room', 'other_occupant_room')
    append_inner_wall(iws, 'inner_wall_main_to_non',   a_iw_mr_nr, 'main_occupant_room', 'non_occupant_room')
    append_inner_wall(iws, 'inner_wall_other_to_non',  a_iw_or_nr, 'other_occupant_room', 'non_occupant_room')

    return iws

example

In [15]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name' : 'ceiling',
                'evlp_type' : 'ceiling',
                'next_space' : 'outdoor', 
                'direction' : 'top',
                'area' : 60.0,
                'space_type' : 'main_occupant_room',
                'spec' : 'something',
            },
            {
                'name' : 'ceiling',
                'evlp_type' : 'ceiling',
                'next_space' : 'outdoor', 
                'direction' : 'top',
                'area' : 60.0,
                'space_type' : 'main_occupant_room',
                'spec' : 'something',
            },
            {
                'name' : 'floor',
                'evlp_type' : 'floor',
                'direction': 'bottom',
                'area' : 60.0,
                'space_type': 'main_occupant_room',
                'spec' : 'something',
            }
        ],
        'windows': [
            {
                'name' : 'window_top',
                'next_space' : 'outdoor',
                'direction' : 'top',
                'area' : 30.0,
                'space_type' : 'main_occupant_room',
                'spec' : 'something',
            }
        ],
        'earthfloor_centers' : [
            { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
            { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
        ]
    }
}

make_inner_walls(d['envelope'], d['common'])

[{'name': 'inner_wall_main_to_other',
  'area': 54.59388622447443,
  'space_type_1': 'main_occupant_room',
  'space_type_2': 'other_occupant_room',
  'spec': {'layers': [{'name': 'gpb',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.22,
     'thickness': 0.0125,
     'volumetric_specific_heat': 830.0},
    {'name': 'air_layer',
     'thermal_resistance_input_method': 'resistance',
     'thermal_resistance': 0.09,
     'volumetric_specific_heat': 0.0},
    {'name': 'gpb',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.22,
     'thickness': 0.0125,
     'volumetric_specific_heat': 830.0}]}},
 {'name': 'inner_wall_main_to_non',
  'area': 19.020025504219888,
  'space_type_1': 'main_occupant_room',
  'space_type_2': 'non_occupant_room',
  'spec': {'layers': [{'name': 'gpb',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.22,
     'thickness': 0.0125,
     'volumetric_specif

### 3. 全体

In [16]:
def convert(d):
    
    common = d['common']
    envelope = d['envelope']
    
    envelope_out = {}
    
    if ('general_parts' in envelope) == True:
        envelope_out['general_parts'] = copy.deepcopy(envelope['general_parts'])
        
    if ('windows' in envelope) == True:
        envelope_out['windows'] = copy.deepcopy(envelope['windows'])
        
    if ('doors' in envelope) == True:
        envelope_out['doors'] = copy.deepcopy(envelope['doors'])
        
    if ('heat_bridges' in envelope) == True:
        envelope_out['heat_bridges'] = copy.deepcopy(envelope['heat_bridges'])
        
    if ('earthfloor_perimeters' in envelope) == True:
        envelope_out['earthfloor_perimeters'] = copy.deepcopy(envelope['earthfloor_perimeters'])
        
    if ('earthfloor_centers' in envelope) == True:
        envelope_out['earthfloor_centers'] = copy.deepcopy(envelope['earthfloor_centers'])
    
    envelope_out['inner_floors'] = make_inner_floors(envelope, common)
    
    envelope_out['inner_walls'] = make_inner_walls(envelope, common)

    return {
        'common' : copy.deepcopy(common),
        'envelope' : envelope_out
    }


example

In [17]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 30.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts' : [
            {'name':'g1',  'evlp_type':'ceiling', 'next_space':'outdoor',         'direction':'top',    'area':16.95,   'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'g2',  'evlp_type':'ceiling', 'next_space':'outdoor',         'direction':'top',    'area':16.95,   'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'g3',  'evlp_type':'ceiling', 'next_space':'outdoor',         'direction':'top',    'area':33.9,    'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'g4',  'evlp_type':'wall',    'next_space':'outdoor',         'direction':'sw',     'area':10.1575, 'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'g5',  'evlp_type':'wall',    'next_space':'outdoor',         'direction':'sw',     'area':10.1575, 'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'g6',  'evlp_type':'wall',    'next_space':'outdoor',         'direction':'sw',     'area':20.315,  'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'g7',  'evlp_type':'wall',    'next_space':'outdoor',         'direction':'nw',     'area':7.4575,  'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'g8',  'evlp_type':'wall',    'next_space':'outdoor',         'direction':'nw',     'area':7.4575,  'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'g9',  'evlp_type':'wall',    'next_space':'outdoor',         'direction':'nw',     'area':14.915,  'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'g10', 'evlp_type':'wall',    'next_space':'outdoor',         'direction':'ne',     'area':15.9725, 'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'g11', 'evlp_type':'wall',    'next_space':'outdoor',         'direction':'ne',     'area':15.9725, 'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'g12', 'evlp_type':'wall',    'next_space':'outdoor',         'direction':'ne',     'area':31.945,  'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'g13', 'evlp_type':'wall',    'next_space':'outdoor',         'direction':'se',     'area':7.4275,  'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'g14', 'evlp_type':'wall',    'next_space':'outdoor',         'direction':'se',     'area':7.4275,  'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'g15', 'evlp_type':'wall',    'next_space':'outdoor',         'direction':'se',     'area':14.855,  'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'g16', 'evlp_type':'floor',   'next_space':'open_underfloor', 'direction':'bottom', 'area':15.0175, 'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'g17', 'evlp_type':'floor',   'next_space':'open_underfloor', 'direction':'bottom', 'area':15.0175, 'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'g18', 'evlp_type':'floor',   'next_space':'open_underfloor', 'direction':'bottom', 'area':30.035,  'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'g19', 'evlp_type':'floor',   'next_space':'open_underfloor', 'direction':'bottom', 'area':1.1025,  'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'g20', 'evlp_type':'floor',   'next_space':'open_underfloor', 'direction':'bottom', 'area':1.1025,  'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'g21', 'evlp_type':'floor',   'next_space':'open_underfloor', 'direction':'bottom', 'area':2.205,   'space_type':'non_occupant_room',   'spec':'something'},
        ],
        'windows' : [
            {'name':'w1',  'next_space':'outdoor', 'direction':'sw', 'area':7.5625, 'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'w2',  'next_space':'outdoor', 'direction':'sw', 'area':7.5625, 'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'w3',  'next_space':'outdoor', 'direction':'sw', 'area':15.125, 'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'w4',  'next_space':'outdoor', 'direction':'nw', 'area':0.7925, 'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'w5',  'next_space':'outdoor', 'direction':'nw', 'area':0.7925, 'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'w6',  'next_space':'outdoor', 'direction':'nw', 'area':1.585,  'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'w7',  'next_space':'outdoor', 'direction':'ne', 'area':1.21,   'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'w8',  'next_space':'outdoor', 'direction':'ne', 'area':1.21,   'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'w9',  'next_space':'outdoor', 'direction':'ne', 'area':2.42,   'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'w10', 'next_space':'outdoor', 'direction':'se', 'area':1.4575, 'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'w11', 'next_space':'outdoor', 'direction':'se', 'area':1.4575, 'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'w12', 'next_space':'outdoor', 'direction':'se', 'area':2.915,  'space_type':'non_occupant_room',   'spec':'something'},
        ],
        'doors' : [
            {'name':'d1', 'next_space':'outdoor', 'direction':'nw', 'area':0.63, 'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'d2', 'next_space':'outdoor', 'direction':'nw', 'area':0.63, 'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'d3', 'next_space':'outdoor', 'direction':'nw', 'area':1.26, 'space_type':'non_occupant_room',   'spec':'something'},
            {'name':'d4', 'next_space':'outdoor', 'direction':'ne', 'area':0.54, 'space_type':'main_occupant_room',  'spec':'something'},
            {'name':'d5', 'next_space':'outdoor', 'direction':'ne', 'area':0.54, 'space_type':'other_occupant_room', 'spec':'something'},
            {'name':'d6', 'next_space':'outdoor', 'direction':'ne', 'area':1.08, 'space_type':'non_occupant_room',   'spec':'something'},
        ],
        'earthfloor_perimeters' : [
            {'name':'ep1', 'next_space':'outdoor',         'direction':'nw',         'length':2.43, 'space_type':'underfloor', 'spec':'something'},
            {'name':'ep2', 'next_space':'outdoor',         'direction':'ne',         'length':1.83, 'space_type':'underfloor', 'spec':'something'},
            {'name':'ep3', 'next_space':'open_underfloor', 'direction':'horizontal', 'length':4.25, 'space_type':'underfloor', 'spec':'something'},
        ]
    }
}

convert(d)

{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 30.0,
  'total_floor_area': 120.0},
 'envelope': {'general_parts': [{'name': 'g1',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'main_occupant_room',
    'spec': 'something'},
   {'name': 'g2',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'other_occupant_room',
    'spec': 'something'},
   {'name': 'g3',
    'evlp_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 33.9,
    'space_type': 'non_occupant_room',
    'spec': 'something'},
   {'name': 'g4',
    'evlp_type': 'wall',
    'next_space': 'outdoor',
    'direction': 'sw',
    'area': 10.1575,
    'space_type': 'main_occupant_room',
    'spec': 'something'},
   {'name': 'g5',
    'evlp_type': 'wall',
    'next_space': 'outdoor',
    'direction': 'sw',
    '